In [71]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [72]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import tweepy
import json
from timeit import default_timer as timer
from tweepy import OAuthHandler

In [73]:
## copy paste these
consumer_key=''
consumer_secret=''
access_token=''
access_token_secret=''

In [74]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

# I. Content

# 1. Gathering Data

In [75]:
screen_name='mindbodygreen' #user name
count=200    #max=200
page=0       #page 0 represent most recent tweets
tweets=api.user_timeline(screen_name=screen_name,count=count,page=0)

In [76]:
# scraping pages from 0 to 29
pages = np.arange(30)

txts = []
for page in pages: 
    tweets = api.user_timeline(screen_name=screen_name, count=count, page=page)
    for tweet in tweets: 
        txts.append(json.loads(json.dumps(tweet._json)))

len(txts)

3414

In [77]:
txts[3413]

{'created_at': 'Wed Nov 27 22:01:01 +0000 2019',
 'id': 1199810385127313408,
 'id_str': '1199810385127313408',
 'text': 'Your favorite, kid-approved side dish just got a low-carb makeover. 😋 https://t.co/qgMk7pxFSV',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/qgMk7pxFSV',
    'expanded_url': 'http://ow.ly/xfys30pVyHW',
    'display_url': 'ow.ly/xfys30pVyHW',
    'indices': [70, 93]}]},
 'source': '<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 17781673,
  'id_str': '17781673',
  'name': 'mindbodygreen',
  'screen_name': 'mindbodygreen',
  'location': 'Brooklyn, NY',
  'description': 'connecting soul & science. 💫',
  'url': 'http://t.co/xQ41BHxbiB',
  'entities': {'url': {'urls': [{'url': 'http://t.co/xQ

In [78]:
#get values lists
time = []
twtid = []
text = []
favorite_count = []
retweet_count=[]
hashtags=[]
url=[]

favorite_count=[]
for line in txts:
    time.append(line['created_at'])
    twtid.append(line['id'])
    text.append(line['text'])
    favorite_count.append(line['favorite_count'])
    retweet_count.append(line['retweet_count'])
    hashtags.append(line['entities']['hashtags'])
    url.append(line['entities']['urls'])

In [79]:
pip install preprocessing

Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install tweet-preprocessor

Note: you may need to restart the kernel to use updated packages.


In [81]:
# remove urls and hashtags in text
#!pip install tweet-preprocessor 

import preprocessor as p
p.clean(text[1])

# remove hushtags and urls in all descriptions
text_clean = []
for line in text:
    text_clean.append(p.clean(line))
text_clean[:1]

['We reached out to registered dietitians who explain the health benefits and side effects, plus how to get more in a']

# 2. Assessing Data

In [82]:
# create data frame
df = pd.DataFrame(list(zip(time, twtid, text_clean, favorite_count, retweet_count, hashtags, url)),
                             columns=['created_at', 'tweet_id','text_clean','favorite_count','retweet_count',
                                     'hashtags','url'])
df

,created_at,tweet_id,text_clean,favorite_count,retweet_count,hashtags,url
0,Fri Jul 31 19:01:44 +0000 2020,1289275069370310656,We reached out to registered dietitians who ex...,2,1,[],"[{'url': 'https://t.co/nyAEuWKwWN', 'expanded_..."
1,Fri Jul 31 17:03:42 +0000 2020,1289245366081093632,Happy . Here's how we're celebrating:,9,4,"[{'text': 'NationalAvocadoDay', 'indices': [6,...","[{'url': 'https://t.co/gE60WESbnB', 'expanded_..."
2,Fri Jul 31 15:03:04 +0000 2020,1289215003954831360,Before figuring out how your own body reacts t...,8,3,[],"[{'url': 'https://t.co/ixbJop7ZvT', 'expanded_..."
3,Fri Jul 31 14:03:14 +0000 2020,1289199948404101120,"We caught up with functional medicine leaders,...",9,3,[],"[{'url': 'https://t.co/atohj6rQy9', 'expanded_..."
4,Fri Jul 31 13:02:53 +0000 2020,1289184761349459969,To acknowledge the beginning of harvest season...,11,3,[],"[{'url': 'https://t.co/jceakbNXM7', 'expanded_..."
...,...,...,...,...,...,...,...
3409,Thu Nov 28 06:00:19 +0000 2019,1199931001599860736,"""I fell in love with the texture at first use,...",8,2,[],"[{'url': 'https://t.co/NhvEuTumKX', 'expanded_..."
3410,Thu Nov 28 04:00:10 +0000 2019,1199900768125562880,The common ways your shadow may be sabotaging ...,2,0,[],"[{'url': 'https://t.co/z7FxuXG1nX', 'expanded_..."
3411,Thu Nov 28 02:00:35 +0000 2019,1199870672077500416,Our love affair with all things tech is taking...,13,11,[],"[{'url': 'https://t.co/4nJUUbjAbm', 'expanded_..."
3412,Thu Nov 28 00:00:54 +0000 2019,1199840553552232450,Here are both preventive strategies this exper...,15,7,[],"[{'url': 'https://t.co/c97nBD2MeC', 'expanded_..."


In [83]:
df['text_clean'].value_counts()

Avoid these to make finding your partner way more pleasantand more likely!                                             4
They might sound similar, but they actually have different symptoms and require totally different treatments.          4
When feels a cold coming on, he reaches for the varieties that will fight off the illness completely (o                4
We've been wondering: Are collagen powder and protein powder interchangeable?                                          4
No. : "Boundaries aren't meant to keep people out; they're meant to make it easier for people to get close."           4
                                                                                                                      ..
While crystals won't do all the work for us, theyre an accessible tool for harnessing good energies + reminding us     1
Just as almonds provide nutrition and health benefits to our diets, they are full of health benefits for our strand    1
There's more to tending to your 

In [16]:
df.describe()

,tweet_id,favorite_count,retweet_count
count,3.413000e+03,3413.000000,3413.000000
mean,1.246019e+18,10.918840,3.947260
std,2.685540e+16,8.122424,3.348921
min,1.199810e+18,0.000000,0.000000
25%,1.222701e+18,5.000000,2.000000
50%,1.245728e+18,9.000000,3.000000
75%,1.269676e+18,14.000000,5.000000
max,1.289245e+18,73.000000,28.000000


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 7 columns):
created_at        3414 non-null object
tweet_id          3414 non-null int64
text_clean        3414 non-null object
favorite_count    3414 non-null int64
retweet_count     3414 non-null int64
hashtags          3414 non-null object
url               3414 non-null object
dtypes: int64(3), object(4)
memory usage: 186.8+ KB


# 3. Clean & Store Data

In [85]:
#copy data
clean_df= df.copy()

In [86]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 7 columns):
created_at        3414 non-null object
tweet_id          3414 non-null int64
text_clean        3414 non-null object
favorite_count    3414 non-null int64
retweet_count     3414 non-null int64
hashtags          3414 non-null object
url               3414 non-null object
dtypes: int64(3), object(4)
memory usage: 186.8+ KB


CLEAN HASHTAGS

In [87]:
clean_df['hashtags'].value_counts()

[]                                                           3302
[{'text': 'mbgPodcast', 'indices': [8, 19]}]                   25
[{'text': 'mbgPodcast', 'indices': [11, 22]}]                  14
[{'text': 'mbgpartner', 'indices': [127, 138]}]                 2
[{'text': 'NationalIceCreamDay', 'indices': [5, 25]}]           2
                                                             ... 
[{'text': 'NationalDisabilityDay', 'indices': [14, 36]}]        1
[{'text': 'mbgpartner', 'indices': [117, 128]}]                 1
[{'text': 'InternationalWomensDay', 'indices': [26, 49]}]       1
[{'text': 'mbgPodcast', 'indices': [56, 67]}]                   1
[{'text': 'mbgPodcast', 'indices': [70, 81]}]                   1
Name: hashtags, Length: 67, dtype: int64

In [88]:
import re
import string

In [89]:
#remove '[]' in hashtags
clean_df.hashtags = clean_df.hashtags.replace('[]','')

In [90]:
#convert hashtags into string
clean_df['hashtags']=clean_df['hashtags'].apply(str)

In [91]:
# look behind
re.findall(r"(?<='text':\s\')\w+",clean_df.hashtags[3])

[]

In [92]:
' '.join(re.findall(r"(?<='text':\s\')\w+",clean_df.hashtags[3]))

''

In [93]:
# apply to every rows
clean_df.hashtags=clean_df.hashtags.apply(lambda row: ' '.join(re.findall(r"(?<='text':\s\')\w+",row)))

In [94]:
#check again
clean_df.hashtags.value_counts()

                                      3302
mbgPodcast                              54
mbgpartner                              16
TraderJoes                               8
NationalIceCreamDay                      4
NationalHempDay                          3
NationalAvocadoDay                       2
AmericanPistachios mbgpartner            2
mbgpodcast                               1
BettyWhite                               1
InternationalWomensDay                   1
EarthDay                                 1
aerieReal                                1
ad                                       1
PlasticFreeJuly                          1
Clean15                                  1
EllieKemper                              1
chomps cantstopwontstop mbgpartner       1
mbgPodcasat                              1
BlackBotanistsWeek                       1
NationalDisabilityDay                    1
covergirls                               1
ReneeZellweger                           1
GalentinesD

CONVERT TIME TO RIGHT DATA TYPE & DATETIME

In [95]:
# Convert timestamp datatype to datetime
clean_df['created_at'] = pd.to_datetime(clean_df['created_at'], infer_datetime_format=True)

In [96]:
#Check again
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 7 columns):
created_at        3414 non-null datetime64[ns, UTC]
tweet_id          3414 non-null int64
text_clean        3414 non-null object
favorite_count    3414 non-null int64
retweet_count     3414 non-null int64
hashtags          3414 non-null object
url               3414 non-null object
dtypes: datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 186.8+ KB


In [99]:
#save 1st file
clean_df.to_csv('Clean_data_Mindbodygreen.csv', encoding='utf-8', index=False)

In [100]:
#CHECK AGAIN
new_df = pd.read_csv('Clean_data_Mindbodygreen.csv')
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 7 columns):
created_at        3414 non-null object
tweet_id          3414 non-null int64
text_clean        3414 non-null object
favorite_count    3414 non-null int64
retweet_count     3414 non-null int64
hashtags          112 non-null object
url               3414 non-null object
dtypes: int64(3), object(4)
memory usage: 186.8+ KB


CHECK DUPLICATES

In [101]:
sum(new_df.duplicated())

199

In [102]:
new_df['tweet_id'].value_counts()

1289199948404101120    2
1284020342923710464    2
1286496396590436353    2
1286858738511622144    2
1283869401620590592    2
                      ..
1213098100602081280    1
1267018105919868929    1
1213309119911866368    1
1281725131912630273    1
1234689994192805888    1
Name: tweet_id, Length: 3214, dtype: int64

In [103]:
#remove duplicates
new_df.drop_duplicates(inplace=True)

In [104]:
#check again
sum(new_df.duplicated())

0

In [105]:
#SAVE FINAL CSV FILE
new_df.to_csv('Final_Clean_Mindbodygreen.csv', encoding='utf-8', index=False)

In [106]:
final_df = pd.read_csv('Final_Clean_Mindbodygreen.csv')
final_df

,created_at,tweet_id,text_clean,favorite_count,retweet_count,hashtags,url
0,2020-07-31 19:01:44+00:00,1289275069370310656,We reached out to registered dietitians who ex...,2,1,NaN,"[{'url': 'https://t.co/nyAEuWKwWN', 'expanded_..."
1,2020-07-31 17:03:42+00:00,1289245366081093632,Happy . Here's how we're celebrating:,9,4,NationalAvocadoDay,"[{'url': 'https://t.co/gE60WESbnB', 'expanded_..."
2,2020-07-31 15:03:04+00:00,1289215003954831360,Before figuring out how your own body reacts t...,8,3,NaN,"[{'url': 'https://t.co/ixbJop7ZvT', 'expanded_..."
3,2020-07-31 14:03:14+00:00,1289199948404101120,"We caught up with functional medicine leaders,...",9,3,NaN,"[{'url': 'https://t.co/atohj6rQy9', 'expanded_..."
4,2020-07-31 13:02:53+00:00,1289184761349459969,To acknowledge the beginning of harvest season...,11,3,NaN,"[{'url': 'https://t.co/jceakbNXM7', 'expanded_..."
...,...,...,...,...,...,...,...
3210,2019-11-28 06:00:19+00:00,1199931001599860736,"""I fell in love with the texture at first use,...",8,2,NaN,"[{'url': 'https://t.co/NhvEuTumKX', 'expanded_..."
3211,2019-11-28 04:00:10+00:00,1199900768125562880,The common ways your shadow may be sabotaging ...,2,0,NaN,"[{'url': 'https://t.co/z7FxuXG1nX', 'expanded_..."
3212,2019-11-28 02:00:35+00:00,1199870672077500416,Our love affair with all things tech is taking...,13,11,NaN,"[{'url': 'https://t.co/4nJUUbjAbm', 'expanded_..."
3213,2019-11-28 00:00:54+00:00,1199840553552232450,Here are both preventive strategies this exper...,15,7,NaN,"[{'url': 'https://t.co/c97nBD2MeC', 'expanded_..."


In [107]:
final_df['hashtags'].value_counts()

mbgPodcast                            51
mbgpartner                            16
TraderJoes                             8
NationalHempDay                        3
NationalIceCreamDay                    2
AmericanPistachios mbgpartner          2
chomps cantstopwontstop mbgpartner     1
mbgPodcasat                            1
mbgpodcast                             1
BettyWhite                             1
InternationalWomensDay                 1
ad                                     1
PlasticFreeJuly                        1
Clean15                                1
EllieKemper                            1
EarthDay                               1
ObjectiveWellness                      1
BlackBotanistsWeek                     1
NationalDisabilityDay                  1
NationalAvocadoDay                     1
covergirls                             1
ReneeZellweger                         1
GalentinesDay                          1
LUNYA mbgpartner                       1
MothersDay      

# II. Profile

In [66]:
#asses again
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3213 entries, 0 to 3412
Data columns (total 7 columns):
created_at        3213 non-null object
tweet_id          3213 non-null int64
text_clean        3213 non-null object
favorite_count    3213 non-null int64
retweet_count     3213 non-null int64
hashtags          3213 non-null object
url               3213 non-null object
dtypes: int64(3), object(4)
memory usage: 200.8+ KB


In [36]:
#date_since = "2020-05-29"
mindbodygreen=api.get_user(screen_name='mindbodygreen')

In [37]:
#getting the latest total followers count, favorites count and posts count
follower_count = mindbodygreen.followers_count
favourites_count = mindbodygreen.favourites_count
posts_count = mindbodygreen.statuses_count

In [38]:
#latest count output
print(follower_count)
print(favourites_count)
print(posts_count)

305168
17895
84180
